# Construct a RAG based on a Knowledge Graph out of text using a LLM

1. Which LLM to use?
2. Which Prompt to use to extract nodes and edges?
3. How to Visualize the Graph?
4. Which Graph Database to use?
5. What does similarity mean?


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import sys
import os
import json
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [3]:
###################### some more libraries to use #######################
# !pip install wikipedia        #Load Data from Wikipedia
# !pip install kuzu             #install and start KUZU Graph DB
#                               #see: https://docs.llamaindex.ai/en/stable/examples/index_structs/knowledge_graph/KuzuGraphDemo.html
# !pip install llama-cpp-python #llama.cpp binding
# !pip install llama-index
# !pip install langchain
#
# !pip install openpyxl
#
# networks science
# !pip install networkx
# !pip install seaborn
# !pip install git+https://github.com/WestHealth/pyvis.git
# see: https://pyvis.readthedocs.io/en/latest/tutorial.html

## Load Data into Documents and chunk them

In [4]:
################ Load some data from wikipedia using WikipediaReader from llama_hub #################
from time import time
from llama_index import download_loader
WikipediaReader = download_loader("WikipediaReader")
loader = WikipediaReader()

wikipedia_pages = ['Albert Einstein']

documents = loader.load_data(pages=wikipedia_pages, auto_suggest=False, lang='en')

for document, page in zip(documents, wikipedia_pages):
    document.metadata = {'doc_source': 'wikipedia', 'wikipedia_page': page, 'doc_lang': 'en', 'doc_size': len(document.text), 'timestamp': time()}

In [5]:
print(documents[0].metadata)
print(documents[0].text)

{'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702739441.176578}
Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theor

In [6]:
######### save the document as json
doc_as_json = [document.to_json() for document in documents]
with open("./output/db/documents.json", "w") as f:
    f.write(json.dumps(doc_as_json))

In [7]:
######### chunk the document
import uuid
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)


langchain_docs = [document.to_langchain_format() for document in documents]

chunks = splitter.split_documents(langchain_docs)

def set_chunk_id(chunks):
    for chunk in chunks:
        chunk.metadata['uuid'] = str(uuid.uuid4())

set_chunk_id(chunks)

print("Number of chunks = ", len(chunks))

raw_docs = [doc.text for doc in documents]
raw_chunks =  [chunk.page_content for chunk in chunks]

Number of chunks =  97


In [8]:
print(chunks[0].metadata)
print(chunks[0].page_content)

{'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702739441.176578, 'uuid': 'e1b9b0f1-7b54-4fe7-b475-fc372faa63d4'}
Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a 

In [9]:
######### save the chunks as json
chunks_as_json = [chunk.to_json() for chunk in chunks]
with open("./output/db/chunks.json", "w") as f:
    f.write(json.dumps(chunks_as_json))

## Use a LLM to extract concepts from chunked data

In [10]:
############### which model to use
#model = "llama-2-13b-chat.Q5_K_M.gguf"
model = "mistral-7b-openorca.Q5_K_M.gguf" #https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GGUF
llama_cpp_path = "/Users/done/Documents/dneumnn/llama.cpp/models"
model_path = os.path.join(llama_cpp_path, model)

In [11]:
############### Initialize Large Language Model with Llana.cpp
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt

llm = LlamaCPP(
    model_url=None,
    model_path=model_path,
    temperature=0.1,
    max_new_tokens=2000,
    context_window=4000,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /Users/done/Documents/dneumnn/llama.cpp/models/mistral-7b-openorca.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  4096, 32002,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q5_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q6_K     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q5_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q5_K     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down.weig

In [12]:
#from llama_index.prompts.base import PromptTemplate
#prompt_template = PromptTemplate(SYS_PROMPT)
#prompt = prompt_template.format(max_knowledge_triplets=10, text=documents[0])

def extract_concepts(user_prompt: str, llm, metadata={}):
    SYS_PROMPT = (
        "Your task is extract the key concepts (and non personal entities) mentioned in the given context. "
        "Extract only the most important and atomistic concepts, if  needed break the concepts down to the simpler concepts."
        "Categorize the concepts in one of the following categories: "
        "[event, concept, place, object, document, organisation, condition, misc]\n"
        "Format your output as a list of json with the following format:\n"
        "[\n"
        "   {\n"
        '       "entity": The Concept,\n'
        '       "importance": The concontextual importance of the concept on a scale of 1 to 5 (5 being the highest),\n'
        '       "category": The Type of Concept,\n'
        "   }, \n"
        "{ }, \n"
        "]\n"
    )

    prompt_format = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"
    #prompt_format = "system\n{system_prompt}\nuser\n{user_prompt}\nassistant\n"
    
    prompt = prompt_format.format(system_prompt=SYS_PROMPT, user_prompt=user_prompt)
    
    completion_response = llm.complete(prompt)
    text = completion_response.text
    text = text.strip("\n")

    concepts = {}
    try:
        concepts = json.loads(text)
    except Exception as e:
        print(e)
        print("\n\nERROR ### Here is the buggy response: ", text, "\n\n")
        
    result = [dict(item, **metadata) for item in concepts]

    
    return result

In [13]:
concepts = []
chunk = chunk[0]
response = extract_concepts(chunk.page_content, llm, metadata=chunk.metadata)
concepts.extend(response)


llama_print_timings:        load time =    1392.36 ms
llama_print_timings:      sample time =      18.42 ms /   213 runs   (    0.09 ms per token, 11562.26 tokens per second)
llama_print_timings: prompt eval time =    1686.60 ms /   603 tokens (    2.80 ms per token,   357.52 tokens per second)
llama_print_timings:        eval time =    6817.96 ms /   212 runs   (   32.16 ms per token,    31.09 tokens per second)
llama_print_timings:       total time =    8789.19 ms


In [76]:
print(chunk.page_content)
print("="*80)
print(concepts)

Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world's most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science. In a 1999 poll of 130 leading physicists worldwide by the Br

In [14]:
###### This may take a while ....
from tqdm.notebook import trange, tqdm
concepts = []
for chunk in tqdm(chunks):
    response = extract_concepts(chunk.page_content, llm, metadata=chunk.metadata)
    concepts.extend(response)
print(len(x))

  0%|          | 0/97 [00:00<?, ?it/s]

Llama.generate: prefix-match hit

llama_print_timings:        load time =    1392.36 ms
llama_print_timings:      sample time =      21.47 ms /   251 runs   (    0.09 ms per token, 11692.37 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    7871.62 ms /   251 runs   (   31.36 ms per token,    31.89 tokens per second)
llama_print_timings:       total time =    8203.85 ms
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1392.36 ms
llama_print_timings:      sample time =      27.82 ms /   323 runs   (    0.09 ms per token, 11608.68 tokens per second)
llama_print_timings: prompt eval time =     812.89 ms /   181 tokens (    4.49 ms per token,   222.66 tokens per second)
llama_print_timings:        eval time =   12027.19 ms /   322 runs   (   37.35 ms per token,    26.77 tokens per second)
llama_print_timings:       total time =   13

KeyboardInterrupt: 

In [22]:
for i, concept in enumerate(concepts):
    if concept['category'] == "person":
        print(concept)

{'entity': 'Albert Einstein', 'importance': 5, 'category': 'person', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}
{'entity': 'Adolf Hitler', 'importance': 3, 'category': 'person', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '7f9dbafe-f39b-4ae5-96d1-f58e72415df2'}
{'entity': 'Franklin D. Roosevelt', 'importance': 3, 'category': 'person', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '7f9dbafe-f39b-4ae5-96d1-f58e72415df2'}
{'entity': 'Albert Einstein', 'importance': 5, 'category': 'person', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '10326035-baa9-45c3-ad5f-58de2098b98a

In [23]:
with open("./output/db/concepts.json", "w") as f:
    f.write(json.dumps(concepts))

In [77]:
with open("./output/db/concepts.json", "r") as f:
    concepts = json.dumps(f.read())

## Use a LLM to extract nodes and edges from chunked data

In [24]:
def extract_nodes_and_edges(user_prompt: str, llm, metadata={}):

    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While traversing through each sentence, Think about the key terms mentioned in it.\n"
            "\tTerms may include object, entity, location, organization, person, \n"
            "\tcondition, acronym, documents, service, concept, etc.\n"
            "\tTerms should be as atomistic as possible\n\n"
        "Thought 2: Think about how these terms can have one on one relation with other terms.\n"
            "\tTerms that are mentioned in the same sentence or the same paragraph are typically related to each other.\n"
            "\tTerms can be related to many other terms\n\n"
        "Thought 3: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    prompt_format = "<|im_start|>system\n{system_prompt}<|im_end|>\n<|im_start|>user\n{user_prompt}<|im_end|>\n<|im_start|>assistant\n"
    #prompt_format = "system\n{system_prompt}\nuser\n{user_prompt}\nassistant\n"
    
    prompt = prompt_format.format(system_prompt=SYS_PROMPT, user_prompt=user_prompt)
    
    completion_response = llm.complete(prompt)
    text = completion_response.text
    text = text.strip("\n")

    nodes = {}
    try:
        nodes = json.loads(text)
    except Exception as e:
        print(e)
        print("\n\nERROR ### Here is the buggy response: ", text, "\n\n")
        
    result = [dict(item, **metadata) for item in nodes]
    
    return result


In [25]:
###### This may take a while ....
from tqdm.notebook import trange, tqdm
nodes_edges = {}
for chunk in tqdm(chunks):
    print(chunk)
    uuid = chunk.metadata["uuid"]
    nodes_edges[uuid] = {}
    nodes_edges[uuid]['content'] = chunk.page_content
    nodes_edges[uuid]['metadata'] = chunk.metadata
    response = extract_nodes_and_edges(chunk.page_content, llm, metadata=chunk.metadata)
    nodes_edges[uuid]['node_edges'] = response


  0%|          | 0/97 [00:00<?, ?it/s]

page_content='Albert Einstein ( EYEN-styne; German: [ˈalbɛɐt ˈʔaɪnʃtaɪn] ; 14 March 1879 – 18 April 1955) was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time. Best known for developing the theory of relativity, Einstein also made important contributions to quantum mechanics, and was thus a central figure in the revolutionary reshaping of the scientific understanding of nature that modern physics accomplished in the first decades of the twentieth century. His mass–energy equivalence formula E = mc2, which arises from relativity theory, has been called "the world\'s most famous equation". He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect", a pivotal step in the development of quantum theory. His work is also known for its influence on the philosophy of science. In a 1999 poll of 130 leading physicists worl

Llama.generate: prefix-match hit

llama_print_timings:        load time =    8142.94 ms
llama_print_timings:      sample time =      42.44 ms /   474 runs   (    0.09 ms per token, 11167.92 tokens per second)
llama_print_timings: prompt eval time =    2067.02 ms /   689 tokens (    3.00 ms per token,   333.33 tokens per second)
llama_print_timings:        eval time =   15234.95 ms /   473 runs   (   32.21 ms per token,    31.05 tokens per second)
llama_print_timings:       total time =   17895.82 ms
Llama.generate: prefix-match hit


page_content='effect, explained Brownian motion, introduced his special theory of relativity—a theory which addressed the inability of classical mechanics to account satisfactorily for the behavior of the electromagnetic field—and demonstrated that if the special theory is correct, mass and energy are equivalent to each other. In 1915, he proposed a general theory of relativity that extended his system of mechanics to incorporate gravitation. A cosmological paper that he published the following year laid out the implications of general relativity for the modeling of the structure and evolution of the universe as a whole. The middle part of his career also saw him making important contributions to statistical mechanics and quantum theory. Especially notable was his work on the quantum physics of radiation, in which light consists of particles, subsequently called photons.' metadata={'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'tim


llama_print_timings:        load time =    8142.94 ms
llama_print_timings:      sample time =      33.47 ms /   379 runs   (    0.09 ms per token, 11324.59 tokens per second)
llama_print_timings: prompt eval time =     712.75 ms /   181 tokens (    3.94 ms per token,   253.95 tokens per second)
llama_print_timings:        eval time =   12845.98 ms /   378 runs   (   33.98 ms per token,    29.43 tokens per second)
llama_print_timings:       total time =   14028.00 ms
Llama.generate: prefix-match hit


...


llama_print_timings:        load time =    8142.94 ms
llama_print_timings:      sample time =      28.47 ms /   290 runs   (    0.10 ms per token, 10187.23 tokens per second)
llama_print_timings: prompt eval time =    1113.15 ms /   300 tokens (    3.71 ms per token,   269.50 tokens per second)
llama_print_timings:        eval time =   10067.11 ms /   289 runs   (   34.83 ms per token,    28.71 tokens per second)
llama_print_timings:       total time =   11537.95 ms
Llama.generate: prefix-match hit


page_content='== Awards and honors ==\n\nEinstein received numerous awards and honors, and in 1922, he was awarded the 1921 Nobel Prize in Physics "for his services to Theoretical Physics, and especially for his discovery of the law of the photoelectric effect". None of the nominations in 1921 met the criteria set by Alfred Nobel, so the 1921 prize was carried forward and awarded to Einstein in 1922.Einsteinium, one of the synthetic elements in the periodic table, was named in his honor.\n\n\n== Publications ==\n\n\n=== Scientific ===\n\n\n=== Others ===\n\n\n== See also ==\n\n\n== Notes ==\n\n\n== References ==\n\n\n=== Works cited ===\n\n\n== Further reading ==\n\n\n== External links ==' metadata={'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '1d96c770-8f60-4850-8c20-075fefa6cfee'}



llama_print_timings:        load time =    8142.94 ms
llama_print_timings:      sample time =       9.17 ms /   102 runs   (    0.09 ms per token, 11124.44 tokens per second)
llama_print_timings: prompt eval time =     820.15 ms /   212 tokens (    3.87 ms per token,   258.49 tokens per second)
llama_print_timings:        eval time =    3442.42 ms /   101 runs   (   34.08 ms per token,    29.34 tokens per second)
llama_print_timings:       total time =    4380.67 ms
Llama.generate: prefix-match hit


page_content='=== Others ===\n\n\n== See also ==\n\n\n== Notes ==\n\n\n== References ==\n\n\n=== Works cited ===\n\n\n== Further reading ==\n\n\n== External links ==\n\nAlbert Einstein at Curlie\nWorks by Albert Einstein at Project Gutenberg\nWorks by or about Albert Einstein at Internet Archive\nWorks by Albert Einstein at LibriVox (public domain audiobooks) \nEinstein\'s Personal Correspondence: Religion, Politics, The Holocaust, and Philosophy Shapell Manuscript Foundation\nFederal Bureau of Investigation file on Albert Einstein\nEinstein and his love of music, Physics World\nAlbert Einstein on Nobelprize.org  including the Nobel Lecture 11 July 1923 Fundamental ideas and problems of the theory of relativity\nAlbert Einstein Archives Online (80,000+ Documents) Archived 11 August 2011 at the Wayback Machine (MSNBC, 19 March 2012)\nEinstein\'s declaration of intention for American citizenship on the World Digital Library\nAlbert Einstein Collection at Brandeis University\nThe Collecte


llama_print_timings:        load time =    8142.94 ms
llama_print_timings:      sample time =      66.33 ms /   714 runs   (    0.09 ms per token, 10764.52 tokens per second)
llama_print_timings: prompt eval time =    1323.31 ms /   366 tokens (    3.62 ms per token,   276.58 tokens per second)
llama_print_timings:        eval time =   25400.15 ms /   713 runs   (   35.62 ms per token,    28.07 tokens per second)
llama_print_timings:       total time =   27692.44 ms


In [26]:
with open("./output/db/einstein_nodes_edges.json", "w") as f:
    f.write(json.dumps(nodes_edges))

In [15]:
with open("./output/db/einstein_nodes_edges.json", "r") as f:
    nodes_edges = json.loads(f.read())

In [16]:
test = [nodes_edges[key]  for key in nodes_edges]
print(test[0]['node_edges'])

[{'node_1': 'Albert Einstein', 'node_2': 'theory of relativity', 'edge': 'developed the theory of relativity', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}, {'node_1': 'Albert Einstein', 'node_2': 'quantum mechanics', 'edge': 'made important contributions to quantum mechanics', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}, {'node_1': 'Albert Einstein', 'node_2': 'mass–energy equivalence formula E=mc2', 'edge': 'developed the mass–energy equivalence formula E=mc2', 'doc_source': 'wikipedia', 'wikipedia_page': 'Albert Einstein', 'doc_lang': 'en', 'doc_size': 83894, 'timestamp': 1702572674.745187, 'uuid': '0af897c6-3391-496e-a93c-a5a18b350f50'}, {'node_1': 'Albert Einstein', 'node_2': 'photoelectric effect', 'edge': 'disc

## Understand Proximity

### Semantic Proximity

In [17]:
################ compute semantic proximity ###############
import pandas as pd
import numpy as np
def semantic_proximity(nodes_edges) -> pd.DataFrame:
    # index, source, target, relation, weight, uuid " 
    data = {'source': [], 'target': [], 'relation': [], 'weight': [], 'uuids':[] }

    for uuid in nodes_edges:
        node_edges_list = nodes_edges[uuid]['node_edges']
        for item in node_edges_list:
            data['source'].append(item['node_1'])
            data['target'].append(item['node_2'])
            data['relation'].append(item.get('edge', np.nan))
            data['weight'].append(1)
            data['uuids'].append([uuid])
            
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame.from_dict(data)
    graph_dataframe = graph_dataframe.replace(" ", np.nan)
    graph_dataframe = graph_dataframe.replace("", np.nan)
    
    graph_dataframe = graph_dataframe.dropna(subset=["source", "target"])
    
    #graph_dataframe["source"] = graph_dataframe["source"].apply(lambda x: x.lower())
    #graph_dataframe["target"] = graph_dataframe["target"].apply(lambda x: x.lower())

    return graph_dataframe

In [18]:
df_sp = semantic_proximity(nodes_edges)
print(len(df_sp))
df_sp.head()

635


,source,target,relation,weight,uuids
0,Albert Einstein,theory of relativity,developed the theory of relativity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
1,Albert Einstein,quantum mechanics,made important contributions to quantum mechanics,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
2,Albert Einstein,mass–energy equivalence formula E=mc2,developed the mass–energy equivalence formula ...,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
3,Albert Einstein,photoelectric effect,discovered the law of the photoelectric effect,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
4,Albert Einstein,quantum theory,took a pivotal step in the development of quan...,1,[0af897c6-3391-496e-a93c-a5a18b350f50]


In [19]:
df_sp.to_csv("./output/db/einstein_semantic_proximity.csv")
df_sp.to_excel("./output/db/einstein_semantic_proximity.xlsx")

### Contextual Proximity

In [20]:
################ compute contextual proximity ###############
#
# look at each chunk and get a list of all concepts (nodes) 
# nodes that occure in the same chunk have contextual proximity
# compute weight, if node - node pairs occure in many chunks.
#
#############################################################
import pandas as pd
import numpy as np
import networkx as nx
def contextual_proximity2Df(nodes_edges) -> pd.DataFrame:

    projection = {}
    
    for uuid in nodes_edges:
        node_edges_list = nodes_edges[uuid]['node_edges']
        nodes = set()
        for item in node_edges_list:
            nodes.add(item['node_1'])
            nodes.add(item['node_2'])
            
        for source in nodes:
            for target in nodes:
                if source != target:
                    _key = (source, target)
                    if _key not in projection:
                        projection[_key] = set()
                    projection[_key].add(uuid)

    print(len(projection))
    
    # index, source, target, relation, weight, uuid " 
    data = {'source': [], 'target': [], 'relation': [], 'weight': [], 'uuids':[] }
    
    for (source, target) in projection:
                     
        data['source'].append(source)
        data['target'].append(target)
        data['relation'].append("contextual proximity")
        data['weight'].append(len(projection[(source, target)]))
        data['uuids'].append(list(projection[(source, target)]))

    
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame.from_dict(data)
    graph_dataframe = graph_dataframe.replace(" ", np.nan)
    graph_dataframe = graph_dataframe.replace("", np.nan)
    
    graph_dataframe = graph_dataframe.dropna(subset=["source", "target"])
    
    #graph_dataframe["source"] = graph_dataframe["source"].apply(lambda x: x.lower())
    #graph_dataframe["target"] = graph_dataframe["target"].apply(lambda x: x.lower())

    return graph_dataframe

In [21]:
df_cp = contextual_proximity2Df(nodes_edges)
df_cp.head()

8480


,source,target,relation,weight,uuids
0,quantum theory,quantum mechanics,contextual proximity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
1,quantum theory,Albert Einstein,contextual proximity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
2,quantum theory,Nobel Prize in Physics,contextual proximity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
3,quantum theory,mass–energy equivalence formula E=mc2,contextual proximity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
4,quantum theory,theory of relativity,contextual proximity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]


In [22]:
### Visualize Proximity with Heatmap

In [23]:
df_cp.to_csv("./output/db/einstein_contextual_proximity.csv")
df_cp.to_excel("./output/db/einstein_contextual_proximity.xlsx")

In [24]:
################ Concat dataframe ################
df = pd.concat([df_sp,df_cp])
print(len(df))
df.head()

9115


,source,target,relation,weight,uuids
0,Albert Einstein,theory of relativity,developed the theory of relativity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
1,Albert Einstein,quantum mechanics,made important contributions to quantum mechanics,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
2,Albert Einstein,mass–energy equivalence formula E=mc2,developed the mass–energy equivalence formula ...,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
3,Albert Einstein,photoelectric effect,discovered the law of the photoelectric effect,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
4,Albert Einstein,quantum theory,took a pivotal step in the development of quan...,1,[0af897c6-3391-496e-a93c-a5a18b350f50]


In [25]:
df.to_csv("./output/db/einstein_graph_data.csv")
df.to_excel("./output/db/einstein_graph_data.xlsx")

## Build a Graph with NetworkX

In [26]:
################ Correct some data in dataframe
# "Albert Einstein" and "Einstein" should be the same node / concept
df = df.replace("Einstein", "Albert Einstein")
df.head()

,source,target,relation,weight,uuids
0,Albert Einstein,theory of relativity,developed the theory of relativity,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
1,Albert Einstein,quantum mechanics,made important contributions to quantum mechanics,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
2,Albert Einstein,mass–energy equivalence formula E=mc2,developed the mass–energy equivalence formula ...,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
3,Albert Einstein,photoelectric effect,discovered the law of the photoelectric effect,1,[0af897c6-3391-496e-a93c-a5a18b350f50]
4,Albert Einstein,quantum theory,took a pivotal step in the development of quan...,1,[0af897c6-3391-496e-a93c-a5a18b350f50]


In [27]:
df.to_csv("./output/db/einstein_graph_data_cleaned.csv")
df.to_excel("./output/db/einstein_graph_data_cleaned.xlsx")

In [28]:
################ Build the graph with networkx #################
nodes = pd.concat([df['source'], df['target']], axis=0).unique()
print(nodes.shape)

G = nx.MultiDiGraph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node),
        title=str(node)
    )

## Add edges to the graph
for index, row in df.iterrows():
    relation_type = "contextual proximity" if row["relation"] == "contextual proximity" else "semantic proximity"
    G.add_edge(
        str(row["source"]),
        str(row["target"]),
        title=str(row["relation"]),
        type=relation_type,
        weight=row['weight'],
        uuid=row['uuids']
    )
print(G)

(720,)
MultiDiGraph with 720 nodes and 9115 edges


In [30]:
# Condense multi directed graph to directed graph
# which proximity should be used?
# first strategy: eliminate contextual proximity, if there is semantic proximity in place

edge_dict = {}

for (source, target, key, data) in G.edges(keys=True, data=True):
    edge_key = (source, target)
    if edge_key not in edge_dict:
        edge_dict[edge_key] = []
    edge_dict[edge_key].append((source, target, key, data))

to_be_deleted = []

for edge_key in edge_dict:
    if len(edge_dict[edge_key]) > 1:
        edges = edge_dict[edge_key]
        # delete all contextual proximity edges
        for (source, target, key, data) in edges:
            if data['type'] == 'contextual proximity':
                to_be_deleted.append((source, target, key))
                #print(source, target, key)

for source, target, key in to_be_deleted:
    G.remove_edge(source, target, key=key)

print(G)

MultiDiGraph with 720 nodes and 8478 edges


In [31]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  19
[['15 January 1906', '1666', '1902–1909', '1905', 'A New Determination of Molecular Dimensions', 'Abba Eban', 'Academic Assistance Council', 'Adolf Hitler', 'Albert Einstein', 'Albert Einstein House', 'Alexis de Tocqueville', 'Alfred Einstein', 'Alfred Kleiner', "Allies' technology", 'America', 'American Academy of Arts and Sciences', 'American Philosophical Society', 'American character', 'American citizen', 'American citizenship', 'Annus Mirabilis papers', 'Anti-Semitism', 'Argentina', 'Argentine scholars', 'Argentine-Germanic Cultural Institution', 'Argovian cantonal school (a gymnasium) in Aarau, Switzerland', 'Asia', 'Asociación Hebraica Argentina', 'Assistant', 'Atomistic constitution of matter', 'Austro-Hungarian Empire', "Beethoven and Mozart's works", "Beethoven's violin sonatas", 'Bell inequality', "Bell's argument", 'Berlin', 'Betty Neumann', 'Bible', 'Bohr', 'Bolsheviks', 'Bondi and Gold', 'Boris Podolsky and Nathan Rosen', 'Bose–Einstein condens

In [32]:
for node in G.nodes():
    print(node, G.nodes[node])
    break

Albert Einstein {'title': 'Albert Einstein'}


In [33]:
################ Adding Communities to the graph #####
import seaborn as sns
import random
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)

for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = min(G.degree[row['node']],100)

In [34]:
for node in G.nodes():
    print(node, G.nodes[node])
    break

Albert Einstein {'title': 'Albert Einstein', 'group': 1, 'color': '#b0db57', 'size': 100}


## Visualize the Knowledge Graph

In [407]:
################ Visualize the graph #################
from pyvis.network import Network

graph_output_directory = "./output/index.html"

net = Network(
    notebook=False,
    cdn_resources="remote",
    height="900px",
    width="100%",
    select_menu=True,
    filter_menu=True,
    bgcolor="#222222", 
    font_color="white"
)

net.from_nx(G)
# net.repulsion(node_distance=150, spring_length=400)
net.force_atlas_2based(central_gravity=0.015, gravity=-31)
# net.barnes_hut(gravity=-18100, central_gravity=5.05, spring_length=380)
net.show_buttons(filter_=["physics"])

net.show(graph_output_directory, notebook=False)

./output/index.html


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
